In [ ]:



import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch import nn
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score
import model
from utils import *
from cnn_model import CNNModel

## 定义交叉验证折数
num_folds = 10

# 初始化列表以存储结果
all_train_acc = []
all_train_loss = []
all_lr = []
all_test_acc = []
all_test_loss = []
all_precision = []
all_recall = []
all_f1 = []

batch_size = 32
num_workers = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers

# 执行 k-fold 交叉验证
dataset_splits = kfold_split_data(root="/home/WuHX/gbx/SMA/DMVCimage", k=num_folds)

for fold in range(num_folds):
    print(f"Running fold {fold + 1}...")

    # 获取当前折的训练集和验证集数据拆分
    train_images_path, train_images_label, val_images_path, val_images_label = dataset_splits[fold]

    # 定义数据转换
    img_size = 224
    data_transform = {
        "train": transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
        ]),
        "val": transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
        ])
    }

    # 创建训练集和验证集
    train_dataset = MyDataSet(images_path=train_images_path,
                              images_class=train_images_label,
                              transform=data_transform["train"])

    val_dataset = MyDataSet(images_path=val_images_path,
                            images_class=val_images_label,
                            transform=data_transform["val"])

    val_num = len(val_dataset)

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True,
                                               pin_memory=True,
                                               num_workers=num_workers,
                                               collate_fn=train_dataset.collate_fn)

    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             pin_memory=True,
                                             num_workers=num_workers,
                                             collate_fn=val_dataset.collate_fn)

    # 定义模型EARFGGRFDRHTG
    net = model.resnet50(num_classes=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)

    optimizer = optim.SGD(net.parameters(), lr=0.001, weight_decay=0.0001)
    lr_scheduler = create_lr_scheduler(optimizer, len(train_loader), 50,
                                       warmup=True, warmup_epochs=10)

    # 训练循环
    epochs = 500
    fold_train_acc = []
    fold_train_loss = []
    fold_lr = []
    fold_test_acc = []
    fold_test_loss = []
    fold_precision = []
    fold_recall = []
    fold_f1 = []

    for epoch in range(epochs):
        weight_decay = 0.0001
        train_loss, train_acc, train_lr = train_one_epoch_classify(net, optimizer, train_loader, device, epoch,
                                                                   lr_scheduler, weight_decay)

        fold_train_acc.append(train_acc)
        fold_train_loss.append(train_loss)
        fold_lr.append(train_lr)

        accu_loss, accu_num, all_pred, all_prob = evaluate_classify(net,
                                                                    val_loader,
                                                                    device,
                                                                    epoch)
        fold_test_acc.append(accu_num)
        fold_test_loss.append(accu_loss)

        # 计算 Precision、Recall 和 F1 得分
        precision = precision_score(val_images_label, all_pred, average='macro')
        recall = recall_score(val_images_label, all_pred, average='macro')
        f1 = f1_score(val_images_label, all_pred, average='macro')
        fold_precision.append(precision)
        fold_recall.append(recall)
        fold_f1.append(f1)

    all_train_acc.append(fold_train_acc)
    all_train_loss.append(fold_train_loss)
    all_lr.append(fold_lr)
    all_test_acc.append(fold_test_acc)
    all_test_loss.append(fold_test_loss)
    all_precision.append(fold_precision)
    all_recall.append(fold_recall)
    all_f1.append(fold_f1)

# 计算所有折的平均结果
avg_train_acc = np.mean(all_train_acc, axis=0)
avg_train_loss = np.mean(all_train_loss, axis=0)
avg_lr = np.mean(all_lr, axis=0)
avg_test_acc = np.mean(all_test_acc, axis=0)
avg_test_loss = np.mean(all_test_loss, axis=0)
avg_precision = np.mean(all_precision, axis=0)
avg_recall = np.mean(all_recall, axis=0)
avg_f1 = np.mean(all_f1, axis=0)
# Take the maximum values for each fold


avg_train_acc_str = ', '.join(str(x) for x in avg_train_acc)
avg_train_loss_str = ', '.join(str(x) for x in avg_train_loss)
avg_lr_str = ', '.join(str(x) for x in avg_lr)
avg_test_acc_str = ', '.join(str(x) for x in avg_test_acc)
avg_test_loss_str = ', '.join(str(x) for x in avg_test_loss)
avg_precision_str = ', '.join(str(x) for x in avg_precision)
avg_recall_str = ', '.join(str(x) for x in avg_recall)
avg_f1_str = ', '.join(str(x) for x in avg_f1)


max_test_acc = np.max(all_test_acc, axis=1)
avg_max_test_acc = np.mean(max_test_acc)

###先取出10折对应的10个列表，然后对10个列表中的对应位置的值取平均值，输出一个（50*1）的列表，可以看出模型效果的平均变化趋势

print("all_test_acc =[", avg_test_acc_str,"]")
print("all_test_loss =[", avg_test_loss_str,"]")
print("all_precision =[", avg_precision_str,"]")
print("all_recall = [", avg_recall_str,"]")
print("all_f1 =[", avg_f1_str,"]")

###先取出10个列表中的最大值，然后对取出的10个最大值取平均值，可以看出模型的最优结果
print("Average Maximum Test Accuracy:", avg_max_test_acc)
